In [1]:
import pandas as pd
import numpy as np
import io
import statsmodels.formula.api as smf
import scipy.stats as stats
import re # эта библиотека необходима для работы с регулярными выражениями

In [25]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import scipy
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_params # вывод результатов тестирования
from statsmodels.iolib.summary2 import summary_col # вывод подгонки

from scipy.stats import t # t-распределение
from scipy.stats import f # F-распределение
import seaborn as sns

In [82]:
alpha = 0.05
t_cr = stats.t.ppf(1 - alpha/2, 569-2 )

In [83]:
date = ''' log(capital)                 0.208***          
                              (0.017)          
                                               
log(labour)                  0.715***          
                              (0.023)          
                                               
Constant                     -1.711***         
                              (0.097)  '''

In [84]:
# Строку с данными можно разбить на 8 подстрок в 3 мя элементами в каждой, разделённых различными количеством пробелом.
# С этим справится метод reshape() из библиотеки numpy
np.reshape(date.strip().split(), (3,3) )

array([['log(capital)', '0.208***', '(0.017)'],
       ['log(labour)', '0.715***', '(0.023)'],
       ['Constant', '-1.711***', '(0.097)']], dtype='<U12')

In [85]:
df = pd.DataFrame(np.reshape(date.strip().split(), (3,3) ), columns=['Coeff','Est str','std.err. str'])
df

,Coeff,Est str,std.err. str
0,log(capital),0.208***,(0.017)
1,log(labour),0.715***,(0.023)
2,Constant,-1.711***,(0.097)


In [86]:
df['Estimate'] = df['Est str'].str.extract(r'([-+]?\d+.\d+)').astype(float)
df['Std.error'] = df['std.err. str'].str.extract(r'([-+]?\d+.\d+)').astype(float)
df

,Coeff,Est str,std.err. str,Estimate,Std.error
0,log(capital),0.208***,(0.017),0.208,0.017
1,log(labour),0.715***,(0.023),0.715,0.023
2,Constant,-1.711***,(0.097),-1.711,0.097


In [87]:
df['lower'] = (df['Estimate'] - df['Std.error']*t_cr).round(2)
df['upper'] = (df['Estimate'] + df['Std.error']*t_cr).round(2)
df

,Coeff,Est str,std.err. str,Estimate,Std.error,lower,upper
0,log(capital),0.208***,(0.017),0.208,0.017,0.17,0.24
1,log(labour),0.715***,(0.023),0.715,0.023,0.67,0.76
2,Constant,-1.711***,(0.097),-1.711,0.097,-1.90,-1.52


In [88]:
# Загрузим данные
df_sleep = pd.read_csv('Labour.csv')
mod1 = smf.ols(formula='np.log(output)~np.log(capital)+np.log(labour)', 
                       data=df_sleep).fit()
# коэффициенты
summary_col(mod1)

,np.log(output)
Intercept,-1.7115
,(0.0967)
np.log(capital),0.2076
,(0.0172)
np.log(labour),0.7148
,(0.0231)
R-squared,0.8378
R-squared Adj.,0.8373


In [89]:
data = '''Intercept	-1.7115
(0.0967)
np.log(capital)	0.2076
(0.0172)
np.log(labour)	0.7148
(0.0231)'''

In [90]:
# Строку с данными можно разбить на 8 подстрок в 3 мя элементами в каждой, разделённых различными количеством пробелом.
# С этим справится метод reshape() из библиотеки numpy
np.reshape(data.strip().split(), (3,3) )

array([['Intercept', '-1.7115', '(0.0967)'],
       ['np.log(capital)', '0.2076', '(0.0172)'],
       ['np.log(labour)', '0.7148', '(0.0231)']], dtype='<U15')

In [91]:
df = pd.DataFrame(np.reshape(data.strip().split(), (3,3) ), columns=['Coeff','Est str','std.err. str'])
df

,Coeff,Est str,std.err. str
0,Intercept,-1.7115,(0.0967)
1,np.log(capital),0.2076,(0.0172)
2,np.log(labour),0.7148,(0.0231)


In [92]:
df['Estimate'] = df['Est str'].str.extract(r'([-+]?\d+.\d+)').astype(float)
df['Std.error'] = df['std.err. str'].str.extract(r'([-+]?\d+.\d+)').astype(float)
df

,Coeff,Est str,std.err. str,Estimate,Std.error
0,Intercept,-1.7115,(0.0967),-1.7115,0.0967
1,np.log(capital),0.2076,(0.0172),0.2076,0.0172
2,np.log(labour),0.7148,(0.0231),0.7148,0.0231


In [93]:
df['lower'] = (df['Estimate'] - df['Std.error']*t_cr).round(2)
df['upper'] = (df['Estimate'] + df['Std.error']*t_cr).round(2)
df

,Coeff,Est str,std.err. str,Estimate,Std.error,lower,upper
0,Intercept,-1.7115,(0.0967),-1.7115,0.0967,-1.90,-1.52
1,np.log(capital),0.2076,(0.0172),0.2076,0.0172,0.17,0.24
2,np.log(labour),0.7148,(0.0231),0.7148,0.0231,0.67,0.76


# Задача 1.3
Постройте 99%-доверительные интервалы для каждого коэффициента(приближённо по результатам таблицы и точно в Python). Ответ округлите до 2-х десятичных знаков.

Для набора данных Electricity рассмотрим линейную регрессию log(cost) на log(q), log^2(q), log(pl), log(pk), log(pf)

In [67]:
# Загрузим данные
df_sleep = pd.read_csv('Electricity.csv')
mod2 = smf.ols(formula='np.log(cost)~np.log(q)+I(np.log(q)**2)+np.log(pl)+np.log(pk)+np.log(pf)', 
                       data=df_sleep).fit()
# коэффициенты
summary_col(mod2)

,np.log(cost)
Intercept,-6.7387
,(0.7063)
np.log(q),0.4030
,(0.0316)
I(np.log(q) ** 2),0.0304
,(0.0022)
np.log(pl),0.1461
,(0.0705)
np.log(pk),0.1571
,(0.0577)


In [75]:
alpha = 0.01
t_cr = stats.t.ppf(1 - alpha/2, 158-5 )

In [76]:
date = '''Intercept	-6.7387
(0.7063)
np.log(q)	0.4030
(0.0316)
I(np.log(q)**2)	0.0304
(0.0022)
np.log(pl)	0.1461
(0.0705)
np.log(pk)	0.1571
(0.0577)
np.log(pf)	0.6847
(0.0427)'''

In [77]:
# Строку с данными можно разбить на 8 подстрок в 3 мя элементами в каждой, разделённых различными количеством пробелом.
# С этим справится метод reshape() из библиотеки numpy
np.reshape(date.strip().split(), (6,3) )

array([['Intercept', '-6.7387', '(0.7063)'],
       ['np.log(q)', '0.4030', '(0.0316)'],
       ['I(np.log(q)**2)', '0.0304', '(0.0022)'],
       ['np.log(pl)', '0.1461', '(0.0705)'],
       ['np.log(pk)', '0.1571', '(0.0577)'],
       ['np.log(pf)', '0.6847', '(0.0427)']], dtype='<U15')

In [78]:
df = pd.DataFrame(np.reshape(date.strip().split(), (6,3) ), columns=['Coeff','Est str','std.err. str'])
df

,Coeff,Est str,std.err. str
0,Intercept,-6.7387,(0.7063)
1,np.log(q),0.4030,(0.0316)
2,I(np.log(q)**2),0.0304,(0.0022)
3,np.log(pl),0.1461,(0.0705)
4,np.log(pk),0.1571,(0.0577)
5,np.log(pf),0.6847,(0.0427)


In [79]:
df['Estimate'] = df['Est str'].str.extract(r'([-+]?\d+.\d+)').astype(float)
df['Std.error'] = df['std.err. str'].str.extract(r'([-+]?\d+.\d+)').astype(float)
df

,Coeff,Est str,std.err. str,Estimate,Std.error
0,Intercept,-6.7387,(0.7063),-6.7387,0.7063
1,np.log(q),0.4030,(0.0316),0.4030,0.0316
2,I(np.log(q)**2),0.0304,(0.0022),0.0304,0.0022
3,np.log(pl),0.1461,(0.0705),0.1461,0.0705
4,np.log(pk),0.1571,(0.0577),0.1571,0.0577
5,np.log(pf),0.6847,(0.0427),0.6847,0.0427


In [80]:
df['lower'] = (df['Estimate'] - df['Std.error']*t_cr).round(2)
df['upper'] = (df['Estimate'] + df['Std.error']*t_cr).round(2)
df

,Coeff,Est str,std.err. str,Estimate,Std.error,lower,upper
0,Intercept,-6.7387,(0.7063),-6.7387,0.7063,-8.58,-4.90
1,np.log(q),0.4030,(0.0316),0.4030,0.0316,0.32,0.49
2,I(np.log(q)**2),0.0304,(0.0022),0.0304,0.0022,0.02,0.04
3,np.log(pl),0.1461,(0.0705),0.1461,0.0705,-0.04,0.33
4,np.log(pk),0.1571,(0.0577),0.1571,0.0577,0.01,0.31
5,np.log(pf),0.6847,(0.0427),0.6847,0.0427,0.57,0.80


In [81]:
# Число наблюдений в модели, число регрессоров и степени свободы для t_cr
mod2.nobs, mod2.df_model, mod2.df_resid

(158.0, 5.0, 152.0)

# ЛУЧШЕЕ РЕШЕНИЕ

In [95]:
sleep75_df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/sleep75.csv')
mod = smf.ols(formula='sleep~totwrk+age+south+male+smsa+yngkid+marr', data=sleep75_df).fit()
mod.summary(slim=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sleep   R-squared:                       0.131
Model:                            OLS   Adj. R-squared:                  0.123
No. Observations:                 706   F-statistic:                     15.06
Covariance Type:            nonrobust   Prob (F-statistic):           2.14e-18
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   3450.9128     80.726     42.748      0.000    3292.418    3609.408
totwrk        -0.1692      0.018     -9.372      0.000      -0.205      -0.134
age            2.6891      1.469      1.830      0.068      -0.195       5.574
south        101.5685     41.837      2.428      0.015      19.427     183.710
male          87.6690     35.104      2.497      0.013      18.747     156.591
smsa         -54.7476     33.123     -1.653      0.099    -119.780      10.285
yngkid       -13.9624     50.341     -0.277      0.782    -112.801      84.876
marr          31.2106     42.233      0.739      0.460     -51.709     114.130
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.26e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [96]:
mod.conf_int(alpha=0.1).round(2)
# Постройте 90%-доверительные интервалы для каждого коэффициента (приближённо по результатам таблицы и точно в Python). Ответ округлите до 2-х десятичных знаков.
# т.е. от 1 отнимаем 0,9 (1-0,9=0,1=alpha)

,0,1
Intercept,3317.95,3583.87
totwrk,-0.20,-0.14
age,0.27,5.11
south,32.66,170.48
male,29.85,145.49
smsa,-109.30,-0.19
yngkid,-96.88,68.95
marr,-38.35,100.77
